In [4]:
from binance.client import Client as bnb_client
import pandas as pd
client = bnb_client(tld='US')

def download_crypto_data(symbols, start_date='2020-01-01', end_date='2024-06-30'):
    client = bnb_client(tld='US')
    
    prices = {}
    for symbol in symbols:
        klines = client.get_historical_klines(symbol, '4h', start_date, end_date)
        df = pd.DataFrame(klines)
        df[0] = pd.to_datetime(df[0], unit='ms')
        df = df.set_index(0)[[4]]
        df.columns = [symbol]
        df[symbol] = df[symbol].astype(float)
        prices[symbol] = df[symbol]
    
    price_df = pd.DataFrame(prices)
    clean_prices = price_df.dropna()
    returns = clean_prices.pct_change().dropna()
    
    return clean_prices, returns


In [93]:
def calculate_sharpe_ratio(strategy_returns):
    if len(strategy_returns) < 30:
        return np.nan
    
    periods_per_year = 2190
    mean_return = strategy_returns.mean()
    std_return = strategy_returns.std()
    
    if std_return == 0:
        return np.nan
        
    return mean_return / std_return * math.sqrt(periods_per_year)

In [5]:
symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'BNBUSDT', 'LINKUSDT']
prices, returns = download_crypto_data(symbols)
print(f"Data shape: {returns.shape}")

Data shape: (5387, 5)


In [90]:
def calculate_reversal_strategy(returns, lookback=1):
    if lookback == 1:
        signal = returns.shift(1)
    else:
        signal = returns.rolling(lookback).mean().shift(1)
    
    ranks = signal.rank(axis=1, method='first')
    n = len(returns.columns)
    weights = (n + 1 - 2*ranks) / n
    weights = weights.subtract(weights.mean(axis=1), axis=0)
    weights = weights.divide(weights.abs().sum(axis=1), axis=0)
    
    return (weights * returns).sum(axis=1).dropna()

In [91]:
def calculate_momentum_strategy(returns, lookback=6):
    signal = returns.rolling(lookback).mean().shift(1)
    ranks = signal.rank(axis=1, method='first')
    n = len(returns.columns)
    weights = (2*ranks - n - 1) / n
    weights = weights.subtract(weights.mean(axis=1), axis=0)
    weights = weights.divide(weights.abs().sum(axis=1), axis=0)
    
    return (weights * returns).sum(axis=1).dropna()